In [ ]:
#importing all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import html
from sklearn.preprocessing import LabelEncoder

nltk.download(['punkt','stopwords'])
stopwords = stopwords.words('english')

# Encoding

In [ ]:
"""df2.loc[df2['rating'] >=7.0,"ratingSentimentLabel"] ="positive"
df2.loc[(df2['rating'] >=4.0) & (df2['rating']<7.0),"ratingSentimentLabel"]= "neutral"
df2.loc[df2['rating']<=3.0,"ratingSentimentLabel"] = "negative""""
#ratingSentimentLabel we need to add this column

In [ ]:
encoder = LabelEncoder()
df2['sentiment_label'] = encoder.fit_transform(df2["vaderSentimentLabel"])
df2['condition'] = encoder.fit_transform(df2["condition"])
df2['drug_name'] = encoder.fit_transform(df2["drug_name"])
df2['sentiment_label'] = encoder.fit_transform(df2["ratingSentimentLabel"])

In [ ]:
df2['useful_count'].mean()

In [ ]:
df2['useful_count_encoded'] = df2['useful_count'].map(lambda x:int(1) if x> 23 else int(0))
#can someone check why my code is not working here, it is working but it is not mapping correctly I'm super dizzy at the moment

In [ ]:
df2['Year_encoded'] = df2['Year'].map(lambda x: int(1) if x >2012 else int(0) )
#so years above 2012 are considered new

In [ ]:
df2.head(10)

In [ ]:
#selecting useful features
df2 = df2[['drug_name','condition','sentiment_label','useful_count_encoded','Year_encoded','sentiment_label']]
#so far I'm still need the limmatized review to do preprocessing, I wanna do some similarity thing

In [ ]:
#df2.to_csv('process2.csv')

# End of Encoding, Madhu Cleaning


In [ ]:
df2.drop(columns=['review','clean_review','rating',])
# I remembered that we might need the ID Columns in order to merge the datasets (mine and the one madhu preprocessed)
#remember this is optional we can apply madhu changes to this

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
test_link = 'https://drive.google.com/uc?id=1J5W75zPGVnf40quZKiZ7d5G8TlKGY88n'
train_link = 'https://drive.google.com/uc?id=1DgVBjs09tUEtLKdrpL24amz4XcqAysvp'

train_df = pd.read_csv(train_link,delimiter='\t')
test_df = pd.read_csv(test_link, delimiter='\t')
df = pd.concat([train_df,test_df])
df.head()

In [ ]:
url = "https://drive.google.com/uc?id=1IHQiNrgzT7MNT0TUvfuOOzbDhGppMq1d"
df = pd.read_csv(url)

df.head(5)

,Unnamed: 0,drug_name,condition,useful_count_encoded,ratingSentiment,Year,nltkScore,TF-IDF
0,0,Valsartan,Left Ventricular Dysfunction,1,2,2012,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,1,Guanfacine,ADHD,1,2,2010,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,2,Lybrel,Birth Control,0,0,2009,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,3,Ortho Evra,Birth Control,0,2,2015,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,4,Buprenorphine / naloxone,Opiate Dependence,1,2,2016,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
#exploring condition column
df.condition.unique()

In [ ]:
#checking the number of unique values in the condition column
len(df.condition.unique())
#but we see that there are a few conditions with the </span> tag and of course there are null value

917

In [ ]:
#we are replacing the</span> tag with 'Not Listed/Othe' and filling null values with 'Not Listed/Othe'
df['condition'] = df['condition'].str.replace(r'\d+</span> users found this comment helpful.$', 'Not Listed/Othe', regex=True)
df['condition'] = df['condition'].fillna('Not Listed/Othe')

In [ ]:
len(df.condition.unique())

837

In [ ]:
#correcting the 'Not Listed/Othe' value
df['condition'] = df['condition'].str.replace('Not Listed/Othe', 'Not Listed/Other', regex=True)

In [ ]:
len(df.condition.unique())
#yay

837

In [ ]:
#now for the review column
#first defining a function to clean the html entities and a function to remove escape characters
def clean_html_entities(text):
    return html.unescape(text)

def no_escape(text):
    return text.replace('\r\n', ' ')

def no_escape2(text):
    return text.replace('\r', ' ')

In [ ]:
#removing the html entities and escape characters
df['clean_review'] = df['review'].apply(clean_html_entities)
df['clean_review'] = df['clean_review'].apply(no_escape)
df['clean_review'] = df['clean_review'].apply(no_escape2)

df.head()

In [ ]:
sorted(df['condition'].unique())

In [ ]:
df['condition'] = df['condition'].replace('ibromyalgia', 'Fibromyalgia')
df['condition'] = df['condition'].replace('Binge Eating Disorde', 'Binge Eating Disorder')
#Benign Essential Trem
df['condition'] = df['condition'].replace('Benign Essential Trem', 'Benign Essential Tremor')
#'Bipolar Disorde'
df['condition'] = df['condition'].replace('Bipolar Disorde', 'Bipolar Disorder')
#Bleeding Disorde
df['condition'] = df['condition'].replace('Bleeding Disorde', 'Bleeding Disorder')
#'Body Dysmorphic Disorde'
df['condition'] = df['condition'].replace('Body Dysmorphic Disorde', 'Body Dysmorphic Disorder')
#'Borderline Personality Disorde'
df['condition'] = df['condition'].replace('Borderline Personality Disorde', 'Borderline Personality Disorder')
#'Brain Tum'
df['condition'] = df['condition'].replace('Brain Tum', 'Brain Tumor')

In [ ]:
#Breast Cance
df['condition'] = df['condition'].replace('Breast Cance', 'Breast Cancer')
#Cance
df['condition'] = df['condition'].replace('Cance', 'Cancer')
#'Colorectal Cance'
df['condition'] = df['condition'].replace('Colorectal Cance', 'Colorectal Cancer')
#Corneal Ulce
df['condition'] = df['condition'].replace('Corneal Ulce', 'Corneal Ulcer')
#Dermatitis Herpeti
df['condition'] = df['condition'].replace('Dermatitis Herpeti', 'Dermatitis Herpetiformis')
#'Dissociative Identity Disorde'
df['condition'] = df['condition'].replace('Dissociative Identity Disorde', 'Dissociative Identity Disorder')
#'Duodenal Ulce'
df['condition'] = df['condition'].replace('Duodenal Ulce', 'Duodenal Ulcer')
#'Endometrial Cance'
df['condition'] = df['condition'].replace('Endometrial Cance', 'Endometrial Cancer')
#'Gastric Cance'
df['condition'] = df['condition'].replace('Gastric Cance', 'Gastric Cancer')

In [ ]:
#'Gastrointestinal Stromal Tum'
df['condition'] = df['condition'].replace('Gastrointestinal Stromal Tum', 'Gastrointestinal Stromal Tumor')
#'Generalized Anxiety Disorde'
df['condition'] = df['condition'].replace('Generalized Anxiety Disorde', 'Generalized Anxiety Disorder')
#'Head and Neck Cance'
df['condition'] = df['condition'].replace('Head and Neck Cance', 'Head and Neck Cancer')
#'Hepatic Tum'
df['condition'] = df['condition'].replace('Hepatic Tum', 'Hepatic Tumor')
#'Herpes Zoste'
df['condition'] = df['condition'].replace('Herpes Zoste', 'Herpes Zoster')
#'Hypoactive Sexual Desire Disorde'
df['condition'] = df['condition'].replace('Hypoactive Sexual Desire Disorde', 'Hypoactive Sexual Desire Disorder')

In [ ]:
#Impetig
df['condition'] = df['condition'].replace('Impetig', 'Impetigo')
#'Major Depressive Disorde'
df['condition'] = df['condition'].replace('Major Depressive Disorde', 'Major Depressive Disorder')
#'Not Listed / Othe'
df['condition'] = df['condition'].replace('Not Listed / Othe', 'Not Listed/Other')
#'Obsessive Compulsive Disorde'
df['condition'] = df['condition'].replace('Obsessive Compulsive Disorde', 'Obsessive Compulsive Disorder')
#'Oppositional Defiant Disorde'
df['condition'] = df['condition'].replace('Oppositional Defiant Disorde', 'Oppositional Defiant Disorder')

In [ ]:
#'Ovarian Cance'
df['condition'] = df['condition'].replace('Ovarian Cance', 'Ovarian Cancer')
#Overactive Bladde
df['condition'] = df['condition'].replace('Overactive Bladde', 'Overactive Bladder')
#'Pain/Feve'
df['condition'] = df['condition'].replace('Pain/Feve', 'Pain/Fever')
#Pancreatic Cance
df['condition'] = df['condition'].replace('Pancreatic Cance', 'Pancreatic Cancer')
#Panic Disorde
df['condition'] = df['condition'].replace('Panic Disorde', 'Panic Disorder')
#Paranoid Disorde
df['condition'] = df['condition'].replace('Paranoid Disorde', 'Paranoid Disorder')

In [ ]:
#Parkinsonian Trem
df['condition'] = df['condition'].replace('Parkinsonian Trem', 'Parkinsonian Tremor')
#Peptic Ulce
df['condition'] = df['condition'].replace('Peptic Ulce', 'Peptic Ulcer')
#Periodic Limb Movement Disorde
df['condition'] = df['condition'].replace('Periodic Limb Movement Disorde', 'Periodic Limb Movement Disorder')
#Post Traumatic Stress Disorde
df['condition'] = df['condition'].replace('Post Traumatic Stress Disorde', 'Post Traumatic Stress Disorder')
#Premenstrual Dysphoric Disorde
df['condition'] = df['condition'].replace('Premenstrual Dysphoric Disorde', 'Premenstrual Dysphoric Disorder')
#Prostate Cance
df['condition'] = df['condition'].replace('Prostate Cance', 'Prostate Cancer')

In [ ]:
#Q Feve
df['condition'] = df['condition'].replace('Q Feve', 'Q Fever')
#Rat-bite Feve
df['condition'] = df['condition'].replace('Rat-bite Feve', 'Rat-bite Fever')
#Schizoaffective Disorde
df['condition'] = df['condition'].replace('Schizoaffective Disorde', 'Schizoaffective Disorder')
#Seasonal Affective Disorde
df['condition'] = df['condition'].replace('Seasonal Affective Disorde', 'Seasonal Affective Disorder')
#Shift Work Sleep Disorde
df['condition'] = df['condition'].replace('Shift Work Sleep Disorde', 'Shift Work Sleep Disorder')
#Skin Cance
df['condition'] = df['condition'].replace('Skin Cance', 'Skin Cancer')
#Social Anxiety Disorde
df['condition'] = df['condition'].replace('Social Anxiety Disorde', 'Social Anxiety Disorder')
#Somat
df['condition'] = df['condition'].replace('Somat', 'Somatoform Pain Disorder')

In [ ]:
#'Somatoform Pain Disorde'
df['condition'] = df['condition'].replace('Somatoform Pain Disorde', 'Somatoform Pain Disorder')
#Stomach Cance
df['condition'] = df['condition'].replace('Stomach Cance', 'Stomach Cancer')
#Stomach Ulce
df['condition'] = df['condition'].replace('Stomach Ulce', 'Stomach Ulcer')
#Temporomandibular Joint Disorde
df['condition'] = df['condition'].replace('Temporomandibular Joint Disorde', 'Temporomandibular Joint Disorder')
#Testicular Cance
df['condition'] = df['condition'].replace('Testicular Cance', 'Testicular Cancer')
#Thyroid Cance
df['condition'] = df['condition'].replace('Thyroid Cance', 'Thyroid Cancer')

In [ ]:
#Tic Disorde
df['condition'] = df['condition'].replace('Tic Disorde', 'Tic Disorder')
#Tinea Versicol
df['condition'] = df['condition'].replace('Tinea Versicol', 'Tinea Versicolor')
#Varicella-Zoste
df['condition'] = df['condition'].replace('Varicella-Zoste', 'Varicella-Zoster')
#Vertig
df['condition'] = df['condition'].replace('Vertig', 'Vertigo')

In [ ]:
#acial Lipoatrophy
df['condition'] = df['condition'].replace('acial Lipoatrophy', 'Facial Lipoatrophy')
#acial Wrinkles
df['condition'] = df['condition'].replace('acial Wrinkles', 'Facial Wrinkles')
#actor IX Deficiency
df['condition'] = df['condition'].replace('actor IX Deficiency', 'Factor IX Deficiency')
#ailure to Thrive
df['condition'] = df['condition'].replace('ailure to Thrive', 'Failure to Thrive')
#amilial Cold Autoinflammatory Syndrome
df['condition'] = df['condition'].replace('amilial Cold Autoinflammatory Syndrome', 'Familial Cold Autoinflammatory Syndrome')
#amilial Mediterranean Feve
df['condition'] = df['condition'].replace('amilial Mediterranean Feve', 'Familial Mediterranean Fever')
#atigue
df['condition'] = df['condition'].replace('atigue', 'Fatigue')
#cal Segmental Glomerulosclerosis
df['condition'] = df['condition'].replace('cal Segmental Glomerulosclerosis', 'Focal Segmental Glomerulosclerosis')

In [ ]:
#emale Infertility
df['condition'] = df['condition'].replace('emale Infertility', 'Female Infertility')
#eve
df['condition'] = df['condition'].replace('eve', 'Fever')
#ge (amlodipine / valsartan
df['condition'] = df['condition'].replace('ge (amlodipine / valsartan', 'High Blood Pressure')
#ge HCT (amlodipine / hydrochlorothiazide / valsartan)
df['condition'] = df['condition'].replace('ge HCT (amlodipine / hydrochlorothiazide / valsartan)', 'High Blood Pressure')
#ibrocystic Breast Disease
df['condition'] = df['condition'].replace('ibrocystic Breast Disease', 'Fibrocystic Breast Disease')
#lic Acid Deficiency
df['condition'] = df['condition'].replace('lic Acid Deficiency', 'Folic Acid Deficiency')
#llicle Stimulation
df['condition'] = df['condition'].replace('llicle Stimulation', 'Follicle Stimulation')
#llicular Lymphoma
df['condition'] = df['condition'].replace('llicular Lymphoma', 'Follicular Lymphoma')
#m Pain Disorde
df['condition'] = df['condition'].replace('m Pain Disorde', 'Somatoform Pain Disorder')
#mance Anxiety
df['condition'] = df['condition'].replace('mance Anxiety', 'Performance Anxiety')

In [ ]:
#me
df['condition'] = df['condition'].replace('me', 'Glioblastoma Multiforme')
#min / pioglitazone)
df['condition'] = df['condition'].replace('min / pioglitazone)', 'Diabetes, Type 2')

In [ ]:
#'min / saxagliptin)'
df['condition'] = df['condition'].replace('min / saxagliptin)', 'Diabetes, Type 2 ')
#'min / rosiglitazone)'
df['condition'] = df['condition'].replace('min / rosiglitazone)', 'Diabetes, Type 2')
#min / sitagliptin)
df['condition'] = df['condition'].replace('min / sitagliptin)', 'Diabetes, Type 2')

In [ ]:
#min)
df['condition'] = df['condition'].replace('min)', 'Diabetes, Type 2')
#mis
df['condition'] = df['condition'].replace('mis', 'Dermatitis Herpetiformis')
#mist (
df['condition'] = df['condition'].replace('mist (', 'COPD')
#moterol / mometasone)
df['condition'] = df['condition'].replace('moterol / mometasone)', 'Asthma, Maintenance')

In [ ]:
#moterol)
df['condition'] = df['condition'].replace('moterol)', 'Asthma, Maintenance')
#mulation) (phenylephrine
df['condition'] = df['condition'].replace('mulation) (phenylephrine', 'Nasal Congestion')
#t Care
df['condition'] = df['condition'].replace('t Care', 'Foot Care')
#t Pac with Cyclobenzaprine (cyclobenzaprine)
df['condition'] = df['condition'].replace('t Pac with Cyclobenzaprine (cyclobenzaprine)', 'Muscle Spasm')
#tic (mycophenolic acid)
df['condition'] = df['condition'].replace('tic (mycophenolic acid)', 'Rejection Prophylaxis')

In [ ]:
#unctional Gastric Disorde
df['condition'] = df['condition'].replace('unctional Gastric Disorde', 'Functional Gastric Disorder')
#ungal Infection Prophylaxis
df['condition'] = df['condition'].replace('ungal Infection Prophylaxis', 'Fungal Infection Prophylaxis')
#ungal Pneumonia
df['condition'] = df['condition'].replace('ungal Pneumonia', 'Fungal Pneumonia')
#zen Shoulde
df['condition'] = df['condition'].replace('zen Shoulde', 'Frozen Shoulder')
#Carcinoid Tum
df['condition'] = df['condition'].replace('Carcinoid Tum', 'Carcinoid Tumor')
#Aggressive Behavi
df['condition'] = df['condition'].replace('Aggressive Behavi', 'Aggressive Behaviour')

In [ ]:
df['condition'].value_counts().get('Not Listed/Other')

1763

In [ ]:
sorted(df.condition.unique())

In [ ]:
len(sorted(df.condition.unique()))

820

In [ ]:
df[df['condition'] == 'Women (minoxidil)']

,Unnamed: 0,drug_name,condition,useful_count_encoded,ratingSentiment,Year,nltkScore,TF-IDF
39610,39854,Minoxidil,Women (minoxidil),0,1,2016,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
df[df['condition'] == 'Women (oxybutynin)']

,Unnamed: 0,drug_name,condition,useful_count_encoded,ratingSentiment,Year,nltkScore,TF-IDF
2973,2988,Oxybutynin,Women (oxybutynin),0,1,2014,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
108330,108953,Oxybutynin,Women (oxybutynin),0,2,2015,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
137696,138483,Oxybutynin,Women (oxybutynin),0,2,2015,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
176651,16338,Oxybutynin,Women (oxybutynin),0,2,2013,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
202844,42676,Oxybutynin,Women (oxybutynin),0,2,2014,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
206169,46023,Oxybutynin,Women (oxybutynin),0,2,2013,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
#Women (oxybutynin)
df['condition'] = df['condition'].replace('Women (oxybutynin)', 'Urinary Frequency')

In [ ]:
#Women (minoxidil)
df['condition'] = df['condition'].replace('Women (minoxidil)', 'Not Listed/Other')

In [ ]:
len(sorted(df.condition.unique()))

819

In [ ]:
sorted(df.condition.unique())

In [ ]:
#mulation) (phenylephrine
df['condition'] = df['condition'].replace('mulation) (phenylephrine', 'Nasal Congestion')

In [ ]:
#ge (amlodipine / valsartan)
df[df['condition'] == 'ge (amlodipine / valsartan)']

In [ ]:
#ge (amlodipine / valsartan)
df['condition'] = df['condition'].replace('ge (amlodipine / valsartan)', 'High Blood Pressure')

In [ ]:
df.to_csv('drug_data_clean2.csv', index=False)

ignore ahmed he is just test primary model